In [1]:
scope=0
(( scope|=1<<2 )) # photos
(( scope|=1<<3 )) # audio
(( scope|=1<<16 )) # offline
(( scope|=1<<17 )) # docs
echo "$scope"

196620


In [2]:
echo "https://oauth.vk.com/authorize?client_id=51915772&response_type=token&scope=$scope&revoke=1"

https://oauth.vk.com/authorize?client_id=51915772&response_type=token&scope=196620&revoke=1


In [3]:
echo "https://id.vk.com/auth?redirect_uri=https%3A%2F%2Foauth.vk.com%2Fblank.html&app_id=51915772&response_type=token&scope=$scope"

https://id.vk.com/auth?redirect_uri=https%3A%2F%2Foauth.vk.com%2Fblank.html&app_id=51915772&response_type=token&scope=196620


In [4]:
echo "https://oauth.vk.com/authorize?client_id=6733081&redirect_uri=blank.html&scope=$scope&response_type=token"

https://oauth.vk.com/authorize?client_id=6733081&redirect_uri=blank.html&scope=196620&response_type=token


In [5]:
TOKEN=$(cat ./token_file)

## Группа методов `users`

### Поиск по произвольному запросу 

In [6]:
curl -Gs https://api.vk.com/method/users.search \
-d v=5.199 \
-d access_token=$TOKEN \
-d q="Матюхин" \
-d count=2 | jq

{
  "response": {
    "count": 11166,
    "items": [
      {
        "id": 517882843,
        "track_code": "78ec291eodqKp252uYCVAapYYnDuTCahLad2_8g1oK8XhKzX_ArGs8H1MBy50JAKk4D4xipPTq8rpHb_xlPSxg",
        "first_name": "Pavel",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 329361482,
        "track_code": "579abdfa9ntxZSxr2f680HaE15pp7pWAF3Dtx20RVOpNzNg3xHGREjIwclWMr7DVQl9BIKzt_Y4Rc-3HY3cmgw",
        "first_name": "Grigorii",
        "last_name": "Matiukhin",
        "can_access_closed": true,
        "is_closed": true
      }
    ]
  }
}


In [7]:
curl -Gs https://api.vk.com/method/users.search \
-d v=5.199 \
-d access_token=$TOKEN \
-d q="Матюхин" \
-d count=2 \
-d fields=about,activities,bdate | jq

{
  "response": {
    "count": 11166,
    "items": [
      {
        "id": 517882843,
        "about": "",
        "activities": "",
        "track_code": "a5ba5ef0kun4gaFZzEt-n48a1Og9TNQi6NVAHwS8PWDAtTrGMP_1gLjQ_TGfTivDtqZxE49NvCzu1kAfCtpPCQ",
        "first_name": "Pavel",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 329361482,
        "bdate": "1.7.2003",
        "about": "",
        "activities": "",
        "track_code": "ad6943aaNOx-K1Iysg-_YU3ycSiwuKgc8hXbWhwewiHut0H1Vr9ThTktX1q0CL8zeU3Y3wO5wBL0FttaEniwSA",
        "first_name": "Grigorii",
        "last_name": "Matiukhin",
        "can_access_closed": true,
        "is_closed": true
      }
    ]
  }
}


In [8]:
user_id=$(
    curl -Gs https://api.vk.com/method/users.search \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d q="Матюхин" \
    -d count=2 \
    -d hometown=Moscow \
    | tee >(jq >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 5,
    "items": [
      {
        "id": 737822617,
        "track_code": "2d3ee983Sb39HLCFAwCaIygPYRHXZRXVtKIrupHmLr6E7IZBs7kqx68SjNFTD5YiG-jM0-QNG9O3oiu095RH",
        "first_name": "Romario",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      },
      {
        "id": 2273004,
        "track_code": "4c3987cfgb_IwSDzyejoymoHEoa5ZPqnZcLOu-Y_6fYgQdFWh6XixZ_ME_PPu-rMWOKIHu0T_rZ_p7y1gA",
        "first_name": "Anton",
        "last_name": "Matyukhin",
        "can_access_closed": true,
        "is_closed": false
      }
    ]
  }
}


In [9]:
echo $user_id

737822617


In [10]:
curl -Gs https://api.vk.com/method/users.getFollowers \
-d v=5.199 \
-d access_token=$TOKEN \
-d user_id=$user_id | jq

{
  "response": {
    "count": 53,
    "items": [
      815091873,
      822181760,
      754106530,
      745694113,
      539075208,
      523236642,
      649006809,
      502260917,
      728506309,
      303910990,
      666814859,
      615886554,
      596120431,
      668126350,
      544358353,
      525547111,
      750344121,
      707008454,
      845983304,
      699889712,
      564451351,
      640235273,
      675933697,
      565035031,
      34657950,
      18054956,
      368076549,
      80735991,
      741238893,
      380952954,
      111102867,
      222911255,
      706907459,
      36948703,
      435164616,
      486948463,
      291509053,
      493342860,
      846004551,
      169611638,
      745215990,
      593718582,
      543511185,
      833518571,
      500275235,
      38443196,
      461649609,
      724147965,
      642032785,
      37670947,
      735008424,
      601572607,
      492845249
    ]
  }
}


## Группа методов `database`

In [11]:
ru_id=$(
    curl -Gs https://api.vk.com/method/database.getCountries \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d code=RU \
    | tee >(jq >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 1,
        "title": "Russia"
      }
    ]
  }
}


In [12]:
echo "$ru_id"

1


In [13]:
curl -Gs https://api.vk.com/method/database.getCities \
-d v=5.199 \
-d access_token=$TOKEN \
-d region_id=$ru_id \
-d count=2 \
-d q==Moscow | jq

{
  "response": {
    "count": 88,
    "items": [
      {
        "id": 1,
        "title": "Moscow"
      },
      {
        "id": 1904692,
        "title": "Moscow",
        "area": "Latah County",
        "region": "Idaho"
      }
    ]
  }
}


In [14]:
city_id=$(
    curl -Gs https://api.vk.com/method/database.getCities \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d q=Moscow | jq ".response.items | map(select(.id == $ru_id))" \
    | tee >(jq >&2) \
    | jq '.[0].id'
)

[
  {
    "id": 1,
    "title": "Moscow"
  }
]


In [15]:
echo $city_id

1


In [16]:
curl -Gs https://api.vk.com/method/database.getSchools \
-d v=5.199 \
-d access_token=$TOKEN \
-d city_id=1 \
-d q=="Школа № 1" | jq '.response.items[0]'

In [17]:
curl -Gs https://api.vk.com/method/database.getUniversities \
-d v=5.199 \
-d access_token=$TOKEN \
-d city_id=1 \
-d q="RUDN" | jq

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 338,
        "title": "РУДН"
      }
    ]
  }
}


## Группа методов `audio`???

## Группа методов `docs`

In [18]:
curl -Gs https://api.vk.com/method/docs.get \
-d v=5.199 \
-d access_token=$TOKEN \
-d owner_id=329361482 \
-d type=4 \
-d count=2 | jq

{
  "response": {
    "count": 7,
    "items": [
      {
        "id": 608960774,
        "owner_id": 329361482,
        "title": "THE SOLUTION.png",
        "size": 11465,
        "ext": "png",
        "date": 1631211540,
        "type": 4,
        "url": "https://vk.com/doc329361482_608960774?hash=RlAyEkkJrZxvRh29KvYGkCZQM9yiyenpp0PXqK4ZPMH&dl=rPjWzydH04kMoZZzRChWG292VFyRYzCbeU4u87PvxtP&api=1&no_preview=1",
        "preview": {
          "photo": {
            "sizes": [
              {
                "src": "https://sun9-62.userapi.com/c520036/u329361482/d34/-3/m_8f8c8839e5.jpg",
                "width": 130,
                "height": 24,
                "type": "m"
              },
              {
                "src": "https://sun9-9.userapi.com/c520036/u329361482/d34/-3/s_8f8c8839e5.jpg",
                "width": 100,
                "height": 19,
                "type": "s"
              },
              {
                "src": "https://sun9-62.userapi.com/c520036/u329361482/

In [19]:
upload_url=$(
    curl -Gs https://api.vk.com/method/docs.getUploadServer \
    -d v=5.199 \
    -d access_token=$TOKEN \
    | tee >(jq >&2) \
    | jq '.response.upload_url'
)

{
  "response": {
    "upload_url": "https://pu.vk.com/c909228/upload_doc.php?act=add_doc&mid=329361482&aid=0&gid=0&type=0&hash=7a3e38e7ed3d94bcef7312e129139049&rhash=f2a1d2e5ba009275c82e39bee8e6530c&api=1"
  }
}


In [20]:
echo "$upload_url"

"https://pu.vk.com/c909228/upload_doc.php?act=add_doc&mid=329361482&aid=0&gid=0&type=0&hash=7a3e38e7ed3d94bcef7312e129139049&rhash=f2a1d2e5ba009275c82e39bee8e6530c&api=1"


In [21]:
file_info=$(
    curl -s -X POST ${upload_url:1:-1} -F file=@./lab3.ipynb \
    | tee >(jq >&2) \
    | jq '.file'
)

{
  "file": "329361482|0|0|909228|f38d2eede7|ipynb|49476|lab3.ipynb|66a13565577351c9b00939c6085bd4fc|102314a0289f9599b7f16335c30a608d||||eyJkaXNrIjoiNTEifQ=="
}


In [22]:
echo "$file_info"

"329361482|0|0|909228|f38d2eede7|ipynb|49476|lab3.ipynb|66a13565577351c9b00939c6085bd4fc|102314a0289f9599b7f16335c30a608d||||eyJkaXNrIjoiNTEifQ=="


In [23]:
curl -Gs https://api.vk.com/method/docs.save \
-d v=5.199 \
-d access_token=$TOKEN \
-d file=${file_info:1:-1} \
-d title=$(openssl rand -base64 12)-lab3.ipynb | jq

{
  "response": {
    "type": "doc",
    "doc": {
      "id": 671963638,
      "owner_id": 329361482,
      "title": "/DKkdhrmRWh0iUs1-lab3.ipynb",
      "size": 49476,
      "ext": "ipynb",
      "date": 1714610492,
      "type": 8,
      "url": "https://vk.com/doc329361482_671963638?hash=aIG8jvoHQqjK7utC5GSsfi3lbLnWfnuOy1tzqdJ2gzk&dl=2Th5O1hUo7M34sgcLIKb3gXYxHPLjEikMZlbAIVYOQo&api=1&no_preview=1",
      "is_unsafe": 0,
      "can_manage": true
    }
  }
}


## Группа методов `photos`

In [24]:
photo_id=$(
    curl -Gs https://api.vk.com/method/photos.getAll \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d owner_id=329361482 \
    -d need_hidden=1 \
    -d photo_sizes=1 \
    -d count=2 \
    | tee >(jq 'del(.response.items.[].sizes[0:-1])' >&2) \
    | jq '.response.items[0].id'
)

{
  "response": {
    "count": 10,
    "items": [
      {
        "album_id": 302557033,
        "date": 1714610457,
        "id": 457241795,
        "owner_id": 329361482,
        "sizes": [
          {
            "height": 510,
            "type": "r",
            "width": 510,
            "url": "https://sun9-29.userapi.com/impg/42-Ka8J-nCeAv38QraDQD7ebij3m9ilUM-J5-g/PSOUFIwFSyQ.jpg?size=510x510&quality=96&sign=afd8b538b7c0e73775260afbad520d3f&c_uniq_tag=PG3ZIdXDXywEy5oJbq9q98NZ7lu0gXkBMKs64QBh7wA&type=album"
          }
        ],
        "text": "3U/fuA0HG AuA3 R-418.jpg",
        "web_view_token": "67bfc84dbca3943538",
        "has_tags": false,
        "hidden": 1
      },
      {
        "album_id": 302557033,
        "date": 1714610364,
        "id": 457241794,
        "owner_id": 329361482,
        "sizes": [
          {
            "height": 510,
            "type": "r",
            "width": 510,
            "url": "https://sun9-29.userapi.com/impg/42-Ka8J-nCeAv38QraDQD7ebi

In [25]:
echo "$photo_id"

457241795


In [26]:
curl -Gs https://api.vk.com/method/photos.getComments \
-d v=5.199 \
-d access_token=$TOKEN \
-d owner_id=329361482 \
-d photo_id=$photo_id | jq

{
  "response": {
    "count": 0,
    "items": []
  }
}


In [27]:
curl https://random.imagecdn.app/512/512 -L -o /tmp/img.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    80  100    80    0     0      9      0  0:00:08  0:00:08 --:--:--    20
100   207  100   207    0     0     23      0  0:00:09  0:00:08  0:00:01     0
100 19251  100 19251    0     0   2118      0  0:00:09  0:00:09 --:--:--  2118


In [28]:
album_id=$(
    curl -Gs https://api.vk.com/method/photos.getAlbums \
    -d v=5.199 \
    -d access_token=$TOKEN \
    | tee >(jq >&2) \
    | jq '.response.items | map(select(.title == "photos")).[0].id'
)

{
  "response": {
    "count": 1,
    "items": [
      {
        "id": 302557033,
        "owner_id": 329361482,
        "size": 6,
        "title": "photos",
        "feed_disabled": 0,
        "feed_has_pinned": 0,
        "created": 1714605876,
        "description": "",
        "can_delete": true,
        "can_include_to_feed": false,
        "is_locked": true,
        "privacy_comment": {
          "category": "friends",
          "lists": {
            "allowed": [],
            "excluded": []
          },
          "owners": {
            "allowed": [],
            "excluded": []
          }
        },
        "privacy_view": {
          "category": "only_me",
          "lists": {
            "allowed": [],
            "excluded": []
          },
          "owners": {
            "allowed": [],
            "excluded": []
          }
        },
        "thumb_id": 457241795,
        "thumb_is_last": 1,
        "updated": 1714610457
      }
    ]
  }
}


In [29]:
echo "$album_id"

302557033


In [30]:
if [ "$album_id" = "null" ]; then
    album_id=$(
        curl -Gs https://api.vk.com/method/photos.createAlbum \
        -d v=5.199 \
        -d access_token=$TOKEN \
        -d title="photos" \
        -d privacy_view=only_me \
        | tee >(jq >&2) \
        | jq '.response.id'
    )
fi

In [31]:
echo "$album_id"

302557033


In [32]:
upload_url=$(
    curl -Gs https://api.vk.com/method/photos.getUploadServer \
    -d v=5.199 \
    -d access_token=$TOKEN \
    -d album_id=$album_id \
    | tee >(jq >&2) \
    | jq '.response.upload_url'
)

{
  "response": {
    "album_id": 302557033,
    "upload_url": "https://pu.vk.com/c907728/ss2307/upload.php?act=do_add&mid=329361482&aid=302557033&gid=0&hash=6a0038b54286ab5a4c842a552cc5a3ec&rhash=ebd7e113fb4e0b774a09c1bc2ee946bf&swfupload=1&api=1",
    "user_id": 329361482
  }
}


In [33]:
echo "$upload_url"

"https://pu.vk.com/c907728/ss2307/upload.php?act=do_add&mid=329361482&aid=302557033&gid=0&hash=6a0038b54286ab5a4c842a552cc5a3ec&rhash=ebd7e113fb4e0b774a09c1bc2ee946bf&swfupload=1&api=1"


In [34]:
photo_info=$(
    curl -s -X POST ${upload_url:1:-1} -F file=@/tmp/img.png \
    | tee >(jq >&2)
)

{
  "server": 907728,
  "photos_list": "[{\"markers_restarted\":true,\"photo\":\"d647a70507:x\",\"sizes\":[],\"latitude\":0,\"longitude\":0,\"kid\":\"87b933148318e860f8e000e4ec16c363\",\"sizes2\":[[\"s\",\"c2db0559dcd95414018b096115a1e2bfbf6ef30bad98d88074f6df3a\",\"-6507193693994561670\",75,75],[\"m\",\"8d25966056b68a844b8ec72534ca7756a1995e6754c532d41eb90c18\",\"-1791684572540499739\",130,130],[\"x\",\"bc1e103eae52f106d5a649b1c87c09db7bf493537cd41923f09f6c91\",\"8007909409160257993\",512,512],[\"o\",\"8d25966056b68a844b8ec72534ca7756a1995e6754c532d41eb90c18\",\"-6568658320281976842\",130,130],[\"p\",\"e876c39ce45567a13b427c53f2287c5127b08ad599ab7b8e11e1f70d\",\"-1325080263667691904\",200,200],[\"q\",\"ea3f95974bc5aee2d1aca0ea27f85278d2b0dad7513a9ac652968daf\",\"8716540838231597853\",320,320],[\"r\",\"1c0a1e73d3713434fdff0ddda451d61a7105b196a22aecf61199ef3f\",\"1501554123802914052\",510,510]],\"urls\":[],\"urls2\":[\"wtsFWdzZVBQBiwlhFaHiv79u8wutmNiAdPbfOg/es_K2M_NsaU.jpg\",\"jSWWYFa2i

In [35]:
server=$(echo "$photo_info" | jq '.server')
photos_list=$(echo "$photo_info" | jq '.photos_list' | sed 's/\\\"/"/g')
hash=$(echo "$photo_info" | jq '.hash')
echo "${hash:1:-1}"

038c2724941ebe35d2139fe7fe3a1dad


In [36]:
curl -Gs https://api.vk.com/method/photos.save \
-d v=5.199 \
-d access_token=$TOKEN \
-d album_id=$album_id \
-d server=$server \
-d photos_list=${photos_list:1:-1} \
-d hash=${hash:1:-1} \
-d caption=$(openssl rand -base64 12)-418.jpg | jq

{
  "response": [
    {
      "album_id": 302557033,
      "date": 1714610506,
      "id": 457241796,
      "owner_id": 329361482,
      "sizes": [
        {
          "height": 75,
          "type": "s",
          "width": 75,
          "url": "https://sun9-7.userapi.com/impg/vB4QPq5S8QbVpkmxyHwJ23v0k1N81Bkj8J9skQ/yWGBDi7PIW8.jpg?size=75x75&quality=96&sign=e79c0cf4be2c0add1ddb574977eefae0&c_uniq_tag=dZmeC6pMP89S5Ac7tspdaM49aL5Je_iD_gWUQpS4LUQ&type=album"
        },
        {
          "height": 130,
          "type": "m",
          "width": 130,
          "url": "https://sun9-7.userapi.com/impg/vB4QPq5S8QbVpkmxyHwJ23v0k1N81Bkj8J9skQ/yWGBDi7PIW8.jpg?size=130x130&quality=96&sign=2d57385d3b579ff4ffec261b3eea7bbc&c_uniq_tag=eLuaZOE8OXS2LmotoKq4Gb7qkO_H7-zJ1A9F2wA2jXc&type=album"
        },
        {
          "height": 512,
          "type": "x",
          "width": 512,
          "url": "https://sun9-7.userapi.com/impg/vB4QPq5S8QbVpkmxyHwJ23v0k1N81Bkj8J9skQ/yWGBDi7PIW8.jpg?size=512x512&qu